In [93]:
import pandas as pd
import re
import nltk
import numpy as np
import string
from collections import Counter
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.collocations import BigramCollocationFinder, TrigramCollocationFinder
from nltk.metrics import BigramAssocMeasures, TrigramAssocMeasures
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans

# Download NLTK resources
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [94]:
# lscs members
lscs = pd.read_csv(
    "lscs_members.csv",
    on_bad_lines="warn"
)

lscs.head()

/tmp/ipython-input-1788044632.py:2: ParserWarning: Skipping line 743: expected 13 fields, saw 15

  lscs = pd.read_csv(


,id,full_name,nickname,email,telegram,position_id,committee_id,college,program,discord,interests,contact_number,fb_link
0,11825782,"Bernaldez, Leo Mikhail B.",NaN,leo_bernaldez@dlsu.edu.ph,NaN,MEM,MEM,GCOE,BS - Chemical Engineering,NaN,NaN,09052898843,https://www.facebook.com/beltran.bernaldez
1,11941820,"De Jesus, Matthew Joseph M.",NaN,matthew.dejesus@dlsu.edu.ph,NaN,MEM,MEM,College of Computer Studies (CCS),BS - Computer Science Major in Network and Inf...,NaN,"Gaming, Editing",0917 618 8375,https://www.facebook.com/matti42/
2,12033391,"Palencia, Ralph Dwanne V",NaN,ralph_palencia@dlsu.edu.ph,NaN,MEM,MEM,SOE,BS Applied Industrial Economics,NaN,"Tech, games",09567349274,fb.com/dwanne.palencia
3,12038660,"Fong, Hannah Regine C.",NaN,hannah_regine_fong@dlsu.edu.ph,NaN,MEM,MEM,CCS,BSMS - Computer Science,NaN,NaN,09177012643,fb.com/fong.hannah
4,12073067,"de Ramos, Ghrazielle Rei A.",NaN,ghrazielle_deramos@dlsu.edu.ph,NaN,MEM,MEM,CCS,BS Computer Science Major in Software Technology,NaN,"Basketball, Sketching, Coding, Volunteering",0933 862 0716,https://web.facebook.com/ghrazielle/


In [95]:
# Define the houses
houses = {
    "Red": {"House Name": "House of Jasper", "Gem/Crystal": "Jasper", "Signature Color/Look": "earthy red/brown-red"},
    "Orange": {"House Name": "House of Carnelian", "Gem/Crystal": "Carnelian", "Signature Color/Look": "fiery orange"},
    "Yellow": {"House Name": "House of Citrine", "Gem/Crystal": "Citrine", "Signature Color/Look": "golden yellow"},
    "Green": {"House Name": "House of Aventurine", "Gem/Crystal": "Aventurine", "Signature Color/Look": "shimmering green"},
    "Light Blue": {"House Name": "House of Amazonite", "Gem/Crystal": "Amazonite", "Signature Color/Look": "turquoise blue-green"},
    "Indigo/Dark Blue": {"House Name": "House of Lapis Lazuli", "Gem/Crystal": "Lapis Lazuli", "Signature Color/Look": "deep royal blue"},
    "Purple": {"House Name": "House of Amethyst", "Gem/Crystal": "Amethyst", "Signature Color/Look": "violet to purple"},
    "White": {"House Name": "House of Moonstone", "Gem/Crystal": "Moonstone", "Signature Color/Look": "milky white with rainbow sheen"},
    "Black": {"House Name": "House of Obsidian", "Gem/Crystal": "Obsidian / Onyx", "Signature Color/Look": "jet black (volcanic glass)"},
    "Pink/Brown": {"House Name": "House of Rose Quartz", "Gem/Crystal": "Serpentine", "Signature Color/Look": "pink"}
}

# Convert to DataFrame
house_df = pd.DataFrame.from_dict(houses, orient="index").reset_index()

# Rename columns
house_df.rename(columns={"index": "Color Category"}, inplace=True)

# Add numeric ID column
house_df.insert(0, "House ID", range(len(house_df)))

# Save to CSV
house_df.to_csv("houses.csv", index=False)

house_df


,House ID,Color Category,House Name,Gem/Crystal,Signature Color/Look
0,0,Red,House of Jasper,Jasper,earthy red/brown-red
1,1,Orange,House of Carnelian,Carnelian,fiery orange
2,2,Yellow,House of Citrine,Citrine,golden yellow
3,3,Green,House of Aventurine,Aventurine,shimmering green
4,4,Light Blue,House of Amazonite,Amazonite,turquoise blue-green
5,5,Indigo/Dark Blue,House of Lapis Lazuli,Lapis Lazuli,deep royal blue
6,6,Purple,House of Amethyst,Amethyst,violet to purple
7,7,White,House of Moonstone,Moonstone,milky white with rainbow sheen
8,8,Black,House of Obsidian,Obsidian / Onyx,jet black (volcanic glass)
9,9,Pink/Brown,House of Rose Quartz,Serpentine,pink


In [96]:
# Clean your "interests" column
lscs["cleaned_interests"] = (
    lscs["interests"]
    .astype(str)
    .replace(r"[&/;\n|*+-]", ",", regex=True)  # added + and -
    .str.replace(r",+", ",", regex=True)        # collapse multiple commas
    .str.strip(", ")                            # strip leading/trailing commas/spaces
)

initial = lscs['cleaned_interests'].sort_values()
initial.to_csv("initial_keywords.csv")
get_nan = lscs[lscs['cleaned_interests'].str.contains('nan',case=False)]
initial

,cleaned_interests
746,1. Coding (I worked on a platform game with tw...
789,3D Modeling and Video Editing
5,?
708,"ACADEMIC,Math, Science, History, Philosophy,MU..."
777,"Anime, Deepwoken, Overwatch, Pokémon, Genshin ..."
...,...
37,what if ayaw ko
56,writing
802,"writing, drawing, tech, games, music"
456,"writing, graphic design, walking"


In [97]:
# Build phrase map automatically from dataframe
stop_words = set(stopwords.words('english'))

# Collect all words from the dataframe
all_words = []
for text in lscs['cleaned_interests']:
    tokens = [w.lower() for w in word_tokenize(str(text)) if w not in string.punctuation and w not in stop_words]
    all_words.extend(tokens)

# Detect top bigrams
bigram_finder = BigramCollocationFinder.from_words(all_words)
top_bigrams = bigram_finder.nbest(BigramAssocMeasures.likelihood_ratio, 20)  # top 20 bigrams

# Detect top trigrams
trigram_finder = TrigramCollocationFinder.from_words(all_words)
top_trigrams = trigram_finder.nbest(TrigramAssocMeasures.likelihood_ratio, 10)  # top 10 trigrams

# Create phrase map: join words with _ so they become single tokens
phrase_map = {" ".join(b): "_".join(b) for b in top_bigrams + top_trigrams}

# Define keyword extraction function with nouns and verbs
def extract_keywords_auto(sentence):
    sentence_lower = str(sentence).lower()

    # Apply phrase mapping
    for phrase, replacement in phrase_map.items():
        if phrase in sentence_lower:
            sentence_lower = sentence_lower.replace(phrase, replacement)

    # Tokenize
    words = word_tokenize(sentence_lower)
    # Remove punctuation and stopwords
    words = [w for w in words if w not in string.punctuation and w not in stop_words]

    # POS tagging
    tagged = pos_tag(words)

    # Keep nouns and verbs
    keywords = [word for word, tag in tagged if tag.startswith('NN') or tag.startswith('VB')]

    # Return unique keywords
    return list(set(keywords))

# Apply to DataFrame
lscs["keywords_list"] = lscs["cleaned_interests"].apply(extract_keywords_auto)

# Save and inspect
keywords = lscs['keywords_list'].sort_values()
keywords.to_csv('keywords.csv', index=False)

# Count empty keyword rows
empty_keywords = lscs['keywords_list'].apply(lambda x: len(x) == 0).sum()
print("Number of empty keyword rows: ", empty_keywords)

# Get rows with empty keywords
empty_rows = lscs[lscs['keywords_list'].apply(lambda x: len(x) == 0)]

# Display their names (or any column you want)
empty_names = empty_rows['full_name'].tolist()
print("Empty keywords found for these names:")
print(empty_names)


Number of empty keyword rows:  2
Empty keywords found for these names:
['Rejano, Hans Martin F.', 'Arous, Adam']


In [98]:
# Convert list of keywords into a comma-separated string
lscs["keywords"] = lscs["keywords_list"].apply(lambda x: ", ".join(x) if x else "")

# Sort the keywords column alphabetically
lscs_sorted = lscs.sort_values(by="keywords")

# Save the sorted CSV
lscs_sorted[['keywords']].to_csv("keywords_sorted.csv", index=False)

# If you want to save the original list column too
lscs_sorted['keywords_list'].to_csv("keywords_list_sorted.csv", index=False)

lscs_sorted

,id,full_name,nickname,email,telegram,position_id,committee_id,college,program,discord,interests,contact_number,fb_link,cleaned_interests,keywords_list,keywords
642,12505196,"Arous, Adam",NaN,adam_arous@dlsu.edu.ph,NaN,MEM,MEM,CCS,BS - Information Technology,NaN,many,0976 216 3937,https://www.facebook.com/adam.arousii?mibextid...,many,[],
5,12107884,"Rejano, Hans Martin F.",NaN,hans_rejano@dlsu.edu.ph,NaN,MEM,MEM,CCS,BS - Computer Science,NaN,?,0929 236 6554,fb.com/rejano03,?,[],
375,12411515,"Fernandez, Bullet Andre F.",NaN,bullet_fernandez@dlsu.edu.ph,NaN,AVP,CORPREL,CCS,BS - Computer Science Major in Software Techno...,NaN,"Programming, Leadership, Journalism, Outdoor A...",0960 340 9380,fb.me/bulletfern,"Programming, Leadership, Journalism, Outdoor A...","[activities, outdoor, programming, journalism,...","activities, outdoor, programming, journalism, ..."
195,12308994,"Valdez, Pulvert M.",NaN,pulvert_valdez@dlsu.edu.ph,NaN,VP,PUBLI,CCS,BS - CSST,NaN,magkacrush sa di ako gusto,09762893125,facebook.com/pulvert.valdez,magkacrush sa di ako gusto,"[ako, di, sa, magkacrush, gusto]","ako, di, sa, magkacrush, gusto"
357,12411035,"Agraviador, Nash Radley V.",NaN,nash_agraviador@dlsu.edu.ph,NaN,MEM,MEM,CCS,BS - Computer Science major in Software Techno...,NaN,"Theatre, Production, Data Analysis",09088698753,https://www.facebook.com/share/1Yj9ekyYsu/,"Theatre, Production, Data Analysis","[analysis, data, production]","analysis, data, production"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
694,12506001,"Mayor, Elijah Rexson B.",NaN,elijah_mayor@dlsu.edu.ph,NaN,MEM,MEM,College of Computer Studies,BS Computer Science major in Network and Infor...,NaN,"Reading, writing, listening to music, biking, ...",0969 640 5538,https://www.facebook.com/elijahrexson.mayor,"Reading, writing, listening to music, biking, ...","[writing, reading, projects, coding, listening...","writing, reading, projects, coding, listening,..."
157,12307513,"Miranda, Juan Miguel B.",NaN,juan_miranda@dlsu.edu.ph,NaN,MEM,MEM,CCS,BS - Computer Studies Major in Software Techno...,NaN,"Games, anime, watching shows and youtube",0976 191 2939,https://www.facebook.com/jm.miranda.858063,"Games, anime, watching shows and youtube","[youtube, games, watching, shows]","youtube, games, watching, shows"
600,12504416,"Chan, Sean Martin Y.",NaN,sean_martin_y_chan@dlsu.edu.ph,NaN,MEM,MEM,CCS,BSCS - Software Technology,NaN,Coding and learning New Programming languages\...,09228908582,https://www.facebook.com/sean.chan.714049,"Coding and learning New Programming languages,...","[youtube, playing, languages, netflix, program...","youtube, playing, languages, netflix, programm..."
156,12307327,"Llanes, Andre Gabriel",NaN,andre_llanes@dlsu.edu.ph,NaN,MEM,MEM,CCS,BS - Computer Science Major in Software Techno...,NaN,"zenless zone zero, dogs",0956 029 6425,https://www.facebook.com/andregabgd/,"zenless zone zero, dogs","[zone, zenless, dogs]","zone, zenless, dogs"


In [99]:
# Flatten and create a DataFrame
keywords_df = lscs['keywords_list'].explode().dropna().to_frame(name="keywords_list")

keywords_df = keywords_df.sort_values(by="keywords_list").reset_index(drop=True)
keywords_df

,keywords_list
0,'m
1,'ve
2,",2."
3,",cybersecurity"
4,",video_games"
...,...
2754,’
2755,’
2756,👍
2757,👽


In [100]:
def clean_keyword(word):
    if not isinstance(word, str):
        return None
    # Lowercase + strip spaces
    word = word.lower().strip()
    # Remove leading/trailing punctuation
    word = re.sub(r'^[\W_]+|[\W_]+$', '', word)
    # Remove if empty or just numbers/punctuation
    if word == "" or re.fullmatch(r"[\W_]+", word) or word.isdigit():
        return None
    return word

# Apply cleaning
keywords_df["keywords_list"] = keywords_df["keywords_list"].apply(clean_keyword)

# Drop None/empty
keywords_df = keywords_df.dropna().drop_duplicates().reset_index(drop=True)
keywords_df = keywords_df.dropna(subset=["keywords_list"])
keywords_df = keywords_df[~keywords_df['keywords_list'].isin(["nan"])].reset_index(drop=True)


print(keywords_df.shape[0])
print(keywords_df.head(20))


749
    keywords_list
0               m
1              ve
2   cybersecurity
3     video_games
4           acads
5          action
6      activities
7     advertising
8      aesthetics
9     aftermarket
10             ai
11            ako
12       analysis
13      analytics
14         animal
15       animanga
16      animation
17          anime
18          animo
19       annoying


In [107]:
model = SentenceTransformer("all-MiniLM-L6-v2")

# Get embeddings for words
embeddings = model.encode(keywords_df["keywords_list"].tolist(), convert_to_tensor=False)

# Cluster on embeddings
n_clusters = 10
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
keywords_df["cluster"] = kmeans.fit_predict(embeddings)

print(keywords_df.sort_values('keywords_list'))
print(keywords_df["cluster"].value_counts())

    keywords_list  cluster
4           acads        7
5          action        0
6      activities        6
7     advertising        5
8      aesthetics        8
..            ...      ...
741         years        1
742       youtube        1
743       zenless        7
744          zone        7
745           ˆᗜˆ        7

[749 rows x 2 columns]
cluster
7    122
1    108
4    107
3     96
5     66
8     65
2     55
0     50
6     48
9     32
Name: count, dtype: int64


In [124]:
clustered_df = (
    keywords_df.groupby("cluster")["keywords_list"]
    .apply(list)
    .reset_index()
    .rename(columns={"keywords_list": "words"})
)

clustered_df

,cluster,words
0,0,"[action, assist, capturing, cleaning, collecti..."
1,1,"[anything, appreciation, binge, bingewatching,..."
2,2,"[video_games, archery, badminton, basketball, ..."
3,3,"[animal, arch, arrangements, baker, baking, bo..."
4,4,"[cybersecurity, ai, applications, architecture..."
5,5,"[advertising, aftermarket, analysis, analytics..."
6,6,"[activities, bike, biking, bouldering, camping..."
7,7,"[m, ve, acads, ako, animanga, animo, annoying,..."
8,8,"[aesthetics, animation, anime, art, arts, astr..."
9,9,"[arranger, band, bass, cello, composing, compo..."
